### Initial Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#  # Descomprimir o ficheiro diretamente para o disco local
#!unzip "/content/drive/MyDrive/deep_learning_project/rare_species 1.zip" -d "/content/"

!unzip "/content/drive/MyDrive/deep_learning_project/dataset_split.zip" -d "/content/"



Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset_split/train/chordata_percidae/2702993_211786_eol-full-size-copy.jpg  
  inflating: /content/dataset_split/train/chordata_percidae/2702994_211786_eol-full-size-copy.jpg  
  inflating: /content/dataset_split/train/chordata_percidae/2702995_211786_eol-full-size-copy.jpg  
  inflating: /content/dataset_split/train/chordata_percidae/2702998_211786_eol-full-size-copy.jpg  
  inflating: /content/dataset_split/train/chordata_percidae/2703001_211786_eol-full-size-copy.jpg  
  inflating: /content/dataset_split/train/chordata_percidae/28479013_211786_eol-full-size-copy.jpg  
  inflating: /content/dataset_split/train/chordata_percidae/29882896_211786_eol-full-size-copy.jpg  
   creating: /content/dataset_split/train/chordata_phasianidae/
  inflating: /content/dataset_split/train/chordata_phasianidae/15267496_45516020_eol-full-size-copy.jpg  
  inflating: /content/dataset_split/train/chordata_phasianidae/15267497_45516

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split


In [4]:
base_path = "/content/dataset_split"

train_ds33 = tf.keras.utils.image_dataset_from_directory(
    directory=f"{base_path}/train",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42
)

val_ds33 = tf.keras.utils.image_dataset_from_directory(
    directory=f"{base_path}/val",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224, 224),
    shuffle=False,
    seed=42
)

test_ds33 = tf.keras.utils.image_dataset_from_directory(
    directory=f"{base_path}/test",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224, 224),
    shuffle=False,
    seed=42
)



Found 8388 files belonging to 202 classes.
Found 1797 files belonging to 202 classes.
Found 1798 files belonging to 202 classes.


In [5]:
n_classes = len(train_ds33.class_names)
print(f"Número de classes: {n_classes}")


Número de classes: 202


In [6]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
    tf.keras.layers.Lambda(lambda x: tf.image.random_brightness(x, max_delta=0.1)),
    tf.keras.layers.Lambda(lambda x: tf.image.random_saturation(x, 0.8, 1.2)),
    tf.keras.layers.Lambda(lambda x: tf.image.random_hue(x, 0.05)),
])


In [7]:
train_ds33 = train_ds33.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE)


In [8]:
train_ds33 = train_ds33.prefetch(tf.data.AUTOTUNE)
val_ds33 = val_ds33.prefetch(tf.data.AUTOTUNE)


In [9]:
import os

base_dir_drive = "/content/drive/MyDrive/deep_learning_project"
image_dir = "/content/rare_species 1"

checkpoint_dir = os.path.join(base_dir_drive, "modelos")

callbacks_finetune_model33 = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, "convnext_finetune.keras"),
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    ),
]

callbacks_head_model33 = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, "convnext_head.keras"),
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    ),
]

In [10]:
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Input
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.regularizers import l2

base_model33 = ConvNeXtBase(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model33.trainable = False

inputs = Input(shape=(224, 224, 3))
x = base_model33(inputs, training=False)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation="gelu", kernel_regularizer=l2(0.002))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation="gelu", kernel_regularizer=l2(0.002))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output = Dense(n_classes, activation="softmax")(x)

model33 = Model(inputs=inputs, outputs=output)

model33.compile(
    optimizer=AdamW(learning_rate=1e-4, weight_decay=3e-4),
    loss=CategoricalCrossentropy(),
    metrics=["accuracy"]
)

history_model33_head = model33.fit(
    train_ds33,
    validation_data=val_ds33,
    epochs=30,
    callbacks=callbacks_head_model33
)

# # unfreeze all
# for layer in base_model33.layers:
#     layer.trainable = True

# model33.compile(
#     optimizer=AdamW(learning_rate=1e-5, weight_decay=3e-4),
#     loss=CategoricalCrossentropy(),
#     metrics=["accuracy"]
# )

# history_model33_finetuned = model33.fit(
#     train_ds33,
#     validation_data=val_ds33,
#     epochs=12,
#     callbacks=callbacks_finetune_model33
# )

350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/30
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.0447 - loss: 9.2769
Epoch 1: val_loss improved from inf to 7.03745, saving model to /content/drive/MyDrive/deep_learning_project/modelos/convnext_head.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 124s 315ms/step - accuracy: 0.0449 - loss: 9.2745 - val_accuracy: 0.3478 - val_loss: 7.0374 - learning_rate: 1.0000e-04
Epoch 2/30
262/263 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.2555 - loss: 7.2475
Epoch 2: val_loss improved from 7.03745 to 5.83866, saving model to /content/drive/MyDrive/deep_learning_project/modelos/convnext_head.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 43s 158ms/step - accuracy: 0.2557 - loss: 7.2459 - val_accuracy: 0.5014 - val_loss: 5.8387 - learning_rate: 1.0000e-04
Epoch 3/30
261/263 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.3640 - loss: 6.3511
Epoch 3: val_loss improved from 5.83866 to 5.31748, saving model to /content/drive/MyDrive/deep_learning_pro

In [ ]:
# model33.save("model_convnext.keras")
# models are already saved with the callbacks


In [11]:
import pickle

base_path = "/content/drive/MyDrive/deep_learning_project"

with open(f"{base_path}/history_convnext_head.pkl", "wb") as f:
    pickle.dump(history_model33_head.history, f)

# with open(f"{base_path}/history_convnext_finetuned.pkl", "wb") as f:
#     pickle.dump(history_model33_finetuned.history, f)
